In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Myriad Pro'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600

import numpy as np
from ipcc_colormap import *

import geopandas as gpd
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from cartopy.io.img_tiles import Stamen
import cartopy.io.shapereader as shpreader
import matplotlib.ticker as mticker
from matplotlib.patches import Circle
from matplotlib.colors import Normalize
import matplotlib.cm as cm
from matplotlib.colorbar import ColorbarBase
from matplotlib import gridspec
from utils import *
import pysteps

import pickle

coastline = gpd.read_file('/home/mizu_home/xp53/nas/home/coastlines-split-SGregion/lines.shp')
mask = np.loadtxt('mask.txt')

ipcc_blue = (112.0/255, 160.0/255, 205.0/255, 1.0)
ipcc_orange = (196.0/255, 121.0/255, 0.0/255, 1.0)

tmp_cmap = ipcc_cmap()
tmp_cmap.read_rgb_data_from_excel()
;

2024-09-03 21:19:55.572183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 21:19:55.656758: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Pysteps configuration file found at: /home/mizu_home/xp53/.conda/envs/wrfplot/lib/python3.8/site-packages/pysteps/pystepsrc



''

In [3]:
# DATA PREPARATION
# the station-based rainfall data is already organized in a N by P matrix
rain_obs = np.loadtxt('data/sta_monthly.csv')

# the simulated rainfall data is reshaped to a 2d matrix of size N by (W x L)
rain_sim_flatten = np.loadtxt('data/wrf_monthly.csv')
rain_sim = rain_sim_flatten.reshape(rain_sim_flatten.shape[0], 120, 160)

# sim_sel constains a P by 4 matrix
# the first two columns are the row and column indices of grids corresponding to the stations
# the last two columns are the lons and lats of grids corresponding to the stations
sim_sel = np.loadtxt('data/wrf_loc.csv')
sim_idx = sim_sel[:, :2].astype(int)

# select simulated rainfall at stations
wrf_sta = np.array([rain_sim[:, i, j] for (i, j) in sim_idx]).T

# lats and lons of all grids are stored in a 2d matrix of 2 by (W x L)
# the first row is lons and the second row is lats
longlat = np.loadtxt('data/lonlat.txt')
lons = longlat[0, :].reshape(120, 160)
lats = longlat[1, :].reshape(120, 160)

# read station lons (3rd column) and lats (4th column)
sta_loc = np.genfromtxt('data/sta_lookup_new.csv', delimiter=',')[:, 2:]


In [ ]:
rain2 = np.zeros((40*12, 120, 160))
# interpolate to all grids
gpr_ = gp_interpolator(P = sim_idx.shape[0])

sg_plotter = sg_map_plotter(lons, lats, coastline)
cmap = tmp_cmap.get_ipcc_cmap('seq', 'prec', 20)

fig = plt.figure(figsize=(11, 10))
gs = gridspec.GridSpec(4, 3, height_ratios=[1,1,1,1], bottom=0.1, top=0.95, left=0.05, right=0.95,
                       wspace=0.05, hspace=0.05)  # Leave space at the bottom for the colorbar

axes = [plt.subplot(gs[i, j], projection=crs.PlateCarree()) for i in range(4) for j in range(3)]

for i, ax in enumerate(axes):
    t1 = time.time()
    gpr_.read_rainfall(rain_obs[i::12, :], wrf_sta[i::12, :])
    gpr_.sn_converge()
    tmp_mu, _ = gpr_.predict(rain_sim_flatten[i::12, :])
    tmp_mu = tmp_mu.T
    tmp_mu = np.reshape(tmp_mu, (40, 120, 160))
    rain2[i::12 ,: ,:] = tmp_mu
    sg_plotter.plot_(ax, np.mean(rain2[i::12, :, :], axis = 0), cmap = cmap, vmin = 0, vmax = 400)
    sg_plotter.plot_scatter(ax, sta_loc, size = 15)
    print(f'{i+1}th month: {time.time() - t1:.2f} sec, the min and max are {np.min(tmp_mu):.2f} and {np.max(tmp_mu):.2f}')

# Adjust the colorbar placement
cbar_ax = fig.add_axes([0.06, 0.07, 0.88, 0.015])  # Position: [left, bottom, width, height]
bounds = np.linspace(0, 400, 21)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.ax.tick_params(labelsize=12)
# make label font larger and bold
cb.set_label('Rainfall Climatology [mm]', fontsize=12, fontweight='bold')

fig.savefig('psd_comp_figures/poster_mean_his.pdf')

output = open('his_rain_interpolations.pkl', 'wb')
pickle.dump(rain2, output)
output.close()


In [4]:
pkl_file = open('his_rain_interpolations.pkl', 'rb')
rain_intp = pickle.load(pkl_file)

In [ ]:
psd, freq = pysteps.utils.spectral.rapsd(np.mean(rain_intp, axis=0), normalize=1, return_freq=True)

In [11]:
psd, freq = pysteps.utils.spectral.rapsd(rain_intp[0, 5:-5, 5:-5], normalize=1, return_freq=True)

fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(10, 12))
# post_r_std = np.zeros((40, 12))
# raw_r_std = np.zeros((40, 12))
for mm in range(12):
    psd_all = np.zeros((40, freq.shape[0]))
    psd_all2 = np.zeros((40, freq.shape[0]))
    
    for i in range(40):
        post_r = rain_intp[12*i+mm, 5:-5, 5:-5]
        raw_r = rain_sim[12*i+mm, 5:-5, 5:-5]
        psd = pysteps.utils.spectral.rapsd(post_r, normalize=1)
        psd2 = pysteps.utils.spectral.rapsd(raw_r, normalize=1)
        psd_all[i, :] = psd
        psd_all2[i, :] = psd
        2
        # post_r_normalized = (post_r - np.min(post_r)) / (np.max(post_r) - np.min(post_r))
        # raw_r_normalized = (raw_r - np.min(raw_r)) / (np.max(raw_r) - np.min(raw_r))
        # post_r_std[i, mm] = np.std(post_r_normalized)
        # raw_r_std[i, mm] = np.std(raw_r_normalized) 
    ax[mm//3, mm%3].plot(0.5 / freq, np.mean(psd_all, axis=0), color = ipcc_blue, label='Posterior')
    ax[mm//3, mm%3].plot(0.5 / freq, np.mean(psd_all2, axis=0), color = ipcc_orange, label='Raw')
    ax[mm//3, mm%3].fill_between(0.5 / freq, np.percentile(psd_all, 5, axis=0), np.percentile(psd_all, 95, axis=0), alpha=0.25, color=ipcc_blue)
    ax[mm//3, mm%3].fill_between(0.5 / freq, np.percentile(psd_all2, 5, axis=0), np.percentile(psd_all2, 95, axis=0), alpha=0.25, color=ipcc_orange)
    ax[mm//3, mm%3].set_xscale('log')
    # ax[mm//3, mm%3].set_ylim(0.005, 0.03)
    ax[mm//3, mm%3].set_title(mm+1)
    # ax[mm//3, mm%3].legend()

# fig, ax = plt.subplots()
# ax.boxplot(post_r_std, positions = np.arange(12) - 0.1, widths = 0.15) 
# ax.boxplot(raw_r_std, positions = np.arange(12) + 0.1, widths = 0.15)
fig.savefig('psd_comp_figures/1d_psd.png')

In [18]:
psd_all

array([[0.0185297 , 0.01832552, 0.01796791, ..., 0.00938418, 0.00917701,
        0.00859177],
       [0.01631463, 0.01611583, 0.01584977, ..., 0.00999988, 0.00984421,
        0.00932953],
       [0.01645547, 0.01605464, 0.01548626, ..., 0.01047855, 0.01035922,
        0.00983458],
       ...,
       [0.01587296, 0.01575676, 0.01547201, ..., 0.01191718, 0.01177577,
        0.01141798],
       [0.0185055 , 0.01826153, 0.01780328, ..., 0.01028131, 0.01017052,
        0.00972609],
       [0.01709484, 0.01703003, 0.0166806 , ..., 0.0107926 , 0.01059193,
        0.01017333]])

In [ ]:
sg_plotter = sg_map_plotter(lons, lats, coastline)
cmap = tmp_cmap.get_ipcc_cmap('seq', 'prec', 20)
fig = plt.figure(figsize=(11, 10))
gs = gridspec.GridSpec(4, 3, height_ratios=[1,1,1,1], bottom=0.1, top=0.95, left=0.05, right=0.95,
                       wspace=0.05, hspace=0.05)  # Leave space at the bottom for the colorbar

axes = [plt.subplot(gs[i, j], projection=crs.PlateCarree()) for i in range(4) for j in range(3)]
rec_mask = np.zeros((120, 160)) 
rec_mask[10:-30, 30:-10] = 100
sg_plotter.plot_(axes[0], rec_mask, cmap = cmap, vmin = 0, vmax = 400)
sg_plotter.plot_scatter(axes[0], sta_loc, size = 15)